In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as f
# import torch.utils.data as data
import torch.optim as optim
from torch.autograd import Variable
import torchvision
import numpy as np
import PIL 

import tensorflow as tf

In [2]:
import os
import sys

MODEL_PATH = '../models'
if MODEL_PATH not in sys.path:
    sys.path.append(MODEL_PATH)

in_dir='../dataset/color/'
img_paths = [x.path for x in os.scandir(in_dir) if x.name.endswith('.jpg') or x.name.endswith('.png')]
len(img_paths)

5578

In [3]:
loader = torchvision.transforms.Compose([
#     transforms.Scale(imsize), 
    torchvision.transforms.ToTensor()
])

def image_loader(image_name):
    """load image, returns cuda tensor"""
    image = PIL.Image.open(image_name)
    image = loader(image).float()
    image = Variable(image, requires_grad=True)
    image = image.unsqueeze(0)  #this is for VGG, may not be needed for ResNet
    return image #assumes that you're using GPU

image = image_loader(img_paths[0])
image=torch.nn.functional.interpolate(image,(image.shape[2]//2, image.shape[3]//2)) # back to original resolution
image.shape

torch.Size([1, 3, 484, 648])

In [4]:
import resnet_detector
model = resnet_detector.DetectorModel()
optimizer = optim.SGD(model.parameters(),lr=0.001)

optimizer.zero_grad()
score_maps_list,endpoints = model(image)

0
1
2


# build_multi_scale_deep_detector_3DNMS

In [5]:
photos=image
batch_size = photos.shape[0]
height = photos.shape[2]
width = photos.shape[3]
C = photos.shape[1]

scale_factors = endpoints['scale_factors']
scale_factors_tensor=torch.tensor(scale_factors)
num_scale = len(score_maps_list)
scale_logits = [None] * num_scale

```
def instance_normalization(inputs):
    # normalize 0-means 1-variance in each sample (not take batch-axis)
    inputs_dim = inputs.get_shape().ndims
    # Epsilon to be used in the tf.nn.batch_normalization
    var_eps = 1e-3    
    if inputs_dim == 4:
        moments_dims = [1,2] # NHWC format
    elif inputs_dim == 2:
        moments_dims = [1]
    else:
        raise ValueError('instance_normalization suppose input dim is 4: inputs_dim={}\n'.format(inputs_dim))
    mean, variance = tf.nn.moments(inputs, axes=moments_dims, keep_dims=True)
    outputs = tf.nn.batch_normalization(inputs, mean, variance, None, None, var_eps) # non-parametric normalization
    return outputs
```
================torch.nn.BatchNorm2d(1, eps=1e-06, momentum=None, affine=None, track_running_stats=None)

In [6]:
instance_normalization=torch.nn.BatchNorm2d(1, eps=1e-06, momentum=None, affine=None, track_running_stats=None)
for i in range(num_scale):
    logits = instance_normalization(score_maps_list[i])
    logits = torch.nn.functional.interpolate(score_maps_list[i],(height, width)) # back to original resolution
    scale_logits[i] = logits
scale_logits=torch.cat(scale_logits, dim=1) # [B,S,H,W]

## def soft_nms_3d(scale_logits, ksize, com_strength=1.0):
    # apply softmax on scalespace logits
    # scale_logits: [B,S,H,W]


In [7]:
num_scales =scale_logits.shape[1]
ksize=15

[B,S,H,W] -> (B,C,S,H,W)--batch size , channel 

In [8]:
scale_logits_d=scale_logits[...,None].permute(0,4,1,2,3)
scale_logits_d.shape

torch.Size([1, 1, 3, 484, 648])

In [9]:
maxpool3d=torch.nn.MaxPool3d((num_scales, ksize, ksize),stride=(num_scales, 1, 1),padding=(0,(ksize-1)//2,(ksize-1)//2),dilation=1)
max_maps=maxpool3d(scale_logits_d)
max_maps = max_maps.squeeze(1) # (B,C,S,H,W) -> [B,S,H,W]
# max_maps = max_maps.permute() # [B,S,H,W]
com_strength=3
exp_maps = torch.exp(com_strength * (scale_logits-max_maps))

In [10]:
exp_maps_d=exp_maps[...,None].permute(0,4,1,2,3)
exp_maps_d.shape

torch.Size([1, 1, 3, 484, 648])

In [11]:
conv3d=torch.nn.Conv3d(1, 1, (num_scales,15,15), stride=(num_scales, 1, 1), padding=(0,(ksize-1)//2,(ksize-1)//2), dilation=1,  bias=False)

In [12]:
sum_ex=conv3d(exp_maps_d)
sum_ex = sum_ex.squeeze(1)
sum_ex.shape

torch.Size([1, 1, 484, 648])

In [13]:
probs = exp_maps / (sum_ex + 1e-6)

In [35]:
probs.shape

torch.Size([1, 3, 484, 648])

## soft_max_and_argmax_1d
(inputs, axis=-1, inputs_index=None, keep_dims=False, com_strength1=250.0, com_strength2=250.0)

In [80]:
def soft_max_and_argmax_1d(inputs, axis=2, inputs_index=None, keep_dims=False, com_strength1=250.0, com_strength2=250.0):
    # Safe softmax
    inputs_max,_=torch.max(inputs, dim=axis, keepdim=keep_dims, out=None)
    inputs_exp1 = torch.exp(com_strength1*(inputs - inputs_max))
    inputs_softmax1_max,_=torch.max(inputs_exp1, dim=axis, keepdim=keep_dims, out=None)
    inputs_softmax1 = inputs_exp1 / (inputs_softmax1_max + 1e-8)

    inputs_exp2 = torch.exp(com_strength2*(inputs - inputs_max))
    inputs_softmax2_max,_=torch.max(inputs_exp2, dim=axis, keepdim=keep_dims, out=None)
    inputs_softmax2 = inputs_exp2 / (inputs_softmax2_max + 1e-8)

    inputs_max,_=torch.max(inputs * inputs_softmax1, dim=axis, keepdim=keep_dims, out=None)
    
    inputs_index_shp = [1,]*(len(inputs.shape)-1)
    inputs_index_shp[axis] = -1
    if inputs_index is None:
        inputs_index = torch.arange(inputs.shape[axis],dtype=inputs.dtype) # use 0,1,2,..,inputs.shape[axis]-1
    print(inputs_index_shp)
    inputs_index = inputs_index.view(1,-1,1).squeeze().unsqueeze(0).unsqueeze(-1).unsqueeze(-1)
    print(inputs_softmax2)
    print(inputs_index)

    inputs_amax = torch.sum(inputs_index.float() * inputs_softmax2, dim=axis, keepdim=keep_dims)
    
    return inputs_max, inputs_amax

In [79]:
scale_factors_tensor.FloatTensor

AttributeError: 'Tensor' object has no attribute 'FloatTensor'

In [81]:
max_heatmaps, max_scales = soft_max_and_argmax_1d(probs, axis=1, 
                                    inputs_index=scale_factors_tensor, keep_dims=False,
                                    com_strength1=100,
                                    com_strength2=100) # both output = [B,H,W]

[1, -1, 1]
tensor([[[[2.6528e-03, 1.0000e+00, 2.5700e-18,  ..., 0.0000e+00,
           3.3165e-35, 0.0000e+00],
          [2.5302e-18, 5.1891e-07, 8.4460e-32,  ..., 3.8302e-40,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 1.0000e+00, 1.5674e-12,  ..., 6.4809e-32,
           0.0000e+00, 0.0000e+00],
          ...,
          [3.6468e-23, 4.7278e-13, 1.1287e-25,  ..., 3.1145e-01,
           1.2950e-03, 9.0395e-07],
          [1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 1.0000e+00,
           1.4257e-14, 6.0391e-01],
          [1.0250e-05, 1.0000e+00, 1.0000e+00,  ..., 1.7135e-03,
           9.5242e-06, 5.6488e-06]],

         [[1.0000e+00, 6.6787e-01, 1.0000e+00,  ..., 1.0000e+00,
           1.0000e+00, 0.0000e+00],
          [1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 1.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 1.0946e-06, 1.0000e+00,  ..., 1.0000e+00,
           1.0000e+00, 0.0000e+00],
          ...,
          [1.0000e+00, 1.3028e-12, 6.7345e-23,  

In [83]:
max_heatmaps.shape

torch.Size([1, 484, 648])

In [85]:
max_scales

tensor([[[0.3562, 1.2361, 0.3536,  ..., 0.3536, 0.3536, 0.1250],
         [0.3536, 0.3536, 0.3536,  ..., 0.3536, 0.1250, 0.1250],
         [0.1250, 1.0000, 0.3536,  ..., 0.3536, 0.3536, 0.1250],
         ...,
         [0.3536, 0.1250, 0.1250,  ..., 0.4905, 0.4118, 0.3548],
         [1.0000, 1.0000, 1.0000,  ..., 1.0374, 0.1257, 0.7290],
         [0.1265, 1.0000, 1.0000,  ..., 0.3553, 0.1250, 0.3536]]],
       grad_fn=<SumBackward2>)

In [45]:
scale_factors_tensor

tensor([1.0000, 0.3536, 0.1250], dtype=torch.float64)

max,max_index=torch.max(x, dim=1, keepdim=True, out=None)
x-max

In [32]:
import gc
gc.collect()

1289